In [1]:
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import torch
import pandas as pd

In [2]:
data = pd.read_csv("Data\\preguntas.csv", sep=',', encoding = "ISO-8859-1")

In [3]:
X = np.load('Data\\X.npy', allow_pickle=True)
y = np.load('Data\\y.npy', allow_pickle=True)

In [4]:
#y_oh = np.zeros((y.shape[0], max(y)+1))
#y_oh[np.arange(y.shape[0]), y] = 1

In [5]:
random.seed(0)
np.random.seed(0)
idx = np.arange(X.shape[0])
random.shuffle(idx)  
btra = np.random.choice(idx, int(0.8*X.shape[0]), replace=False)
btes = [i for i in idx if i not in btra]

In [6]:
Xtra = X[btra]
ytra = y[btra]

Xtes = X[btes]
ytes = y[btes]

In [7]:
Xtes.shape

(228, 611)

In [8]:
def batch_generator(X, Y, n_batches):  
    
    random.seed(123)
    
    batch_size = X.shape[0] // n_batches
    
    idx = np.arange(X.shape[0])
    random.shuffle(idx)    
    idx = idx[:n_batches*batch_size]
        
    for i in range(n_batches):            
        bi = np.random.choice(idx, batch_size, replace=False)
        X_batch = X[bi]
        Y_batch = Y[bi]
        idx = [i for i in idx if i not in bi]
        yield (X_batch,Y_batch)       
        
        
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(611, 24, bias=True)
        self.fc12 = nn.Linear(24, 13, bias=True) 
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = self.fc12(x1)     
        return x1
    
    
def fit(X, Y, Xt, Yt, net, optimizer, error, n_epochs, n_batches, device, PATH, min_val_loss = float('inf')):
    
    net = net.to(device)    
    losses = []    
    val_losses = []

    val_inputs = torch.FloatTensor(Xt)
    val_labels = torch.tensor(Yt, dtype=torch.long)
    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)  
    
    for epoch in range(n_epochs):  # loop over the dataset multiple times
    
        running_loss = 0  
         
        
        for batch_x, batch_y in batch_generator(X, Y, n_batches):  
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # get the inputs
            inputs = torch.FloatTensor(batch_x)
            labels = torch.tensor(batch_y, dtype=torch.long)
            inputs, labels = inputs.to(device), labels.to(device)             
                
    
            # forward + backward + optimize
            outputs = net.forward(inputs)
            loss = error(outputs, labels)
                        
            loss.backward()    #obtain gradients      
            optimizer.step()   #optimize
                
            running_loss += loss.item()      
                
        running_loss = running_loss/n_batches    
        
        with torch.no_grad():
            val_outputs = net.forward(val_inputs)
            val_loss = error(val_outputs, val_labels) 
        
        losses.append(running_loss)   
        val_losses.append(val_loss.item())
        
        
        print('Epoch {0}: Training Loss: {1} Validation Loss: {2}'.format(epoch+1, running_loss, val_loss.item()))
        
        if val_loss.item() < min_val_loss:
            torch.save(net.state_dict(), PATH)
            print('New Checkpoint Saved into PATH')
            min_val_loss = val_loss.item()
        
        
def fweights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n-1)
        #y = 0.0001
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)        
        

In [9]:
n_epochs = 15000
lr = 0.001
n_batches = 3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PATH = 'Data\\model_checkpoint.pt'

In [10]:
weights = np.sum(np.unique(ytes, return_counts=True)[1])/np.unique(ytes, return_counts=True)[1]
class_weights = torch.FloatTensor(weights).cuda()

In [11]:
net = Net1()
net.apply(fweights_init_normal)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

In [12]:
fit(Xtra, ytra, Xtes, ytes, net, optimizer, criterion, n_epochs, n_batches, device, PATH)

Epoch 1: Training Loss: 2.6051472822825112 Validation Loss: 2.599695920944214
New Checkpoint Saved into PATH
Epoch 2: Training Loss: 2.5921079317728677 Validation Loss: 2.5885093212127686
New Checkpoint Saved into PATH
Epoch 3: Training Loss: 2.5698208808898926 Validation Loss: 2.5775885581970215
New Checkpoint Saved into PATH
Epoch 4: Training Loss: 2.5493170420328775 Validation Loss: 2.5686097145080566
New Checkpoint Saved into PATH
Epoch 5: Training Loss: 2.53103240331014 Validation Loss: 2.562180757522583
New Checkpoint Saved into PATH
Epoch 6: Training Loss: 2.5140980084737143 Validation Loss: 2.557875394821167
New Checkpoint Saved into PATH
Epoch 7: Training Loss: 2.503948450088501 Validation Loss: 2.5553739070892334
New Checkpoint Saved into PATH
Epoch 8: Training Loss: 2.4948507150014243 Validation Loss: 2.5541574954986572
New Checkpoint Saved into PATH
Epoch 9: Training Loss: 2.488529364267985 Validation Loss: 2.553579568862915
New Checkpoint Saved into PATH
Epoch 10: Training

Epoch 81: Training Loss: 2.3749120235443115 Validation Loss: 2.479985475540161
New Checkpoint Saved into PATH
Epoch 82: Training Loss: 2.3741981983184814 Validation Loss: 2.4792418479919434
New Checkpoint Saved into PATH
Epoch 83: Training Loss: 2.376454750696818 Validation Loss: 2.4782018661499023
New Checkpoint Saved into PATH
Epoch 84: Training Loss: 2.3714577356974282 Validation Loss: 2.47739577293396
New Checkpoint Saved into PATH
Epoch 85: Training Loss: 2.3709020614624023 Validation Loss: 2.476388931274414
New Checkpoint Saved into PATH
Epoch 86: Training Loss: 2.3719420433044434 Validation Loss: 2.4750142097473145
New Checkpoint Saved into PATH
Epoch 87: Training Loss: 2.368056615193685 Validation Loss: 2.4741556644439697
New Checkpoint Saved into PATH
Epoch 88: Training Loss: 2.3683523337046304 Validation Loss: 2.473055124282837
New Checkpoint Saved into PATH
Epoch 89: Training Loss: 2.366182565689087 Validation Loss: 2.471872091293335
New Checkpoint Saved into PATH
Epoch 90: 

Epoch 162: Training Loss: 2.279041131337484 Validation Loss: 2.393918037414551
New Checkpoint Saved into PATH
Epoch 163: Training Loss: 2.2788195610046387 Validation Loss: 2.392744302749634
New Checkpoint Saved into PATH
Epoch 164: Training Loss: 2.2753336429595947 Validation Loss: 2.3910810947418213
New Checkpoint Saved into PATH
Epoch 165: Training Loss: 2.27768882115682 Validation Loss: 2.38960599899292
New Checkpoint Saved into PATH
Epoch 166: Training Loss: 2.2749008337656655 Validation Loss: 2.38802170753479
New Checkpoint Saved into PATH
Epoch 167: Training Loss: 2.271845897038778 Validation Loss: 2.38708233833313
New Checkpoint Saved into PATH
Epoch 168: Training Loss: 2.270679791768392 Validation Loss: 2.385985851287842
New Checkpoint Saved into PATH
Epoch 169: Training Loss: 2.2706686655680337 Validation Loss: 2.384763240814209
New Checkpoint Saved into PATH
Epoch 170: Training Loss: 2.2686753273010254 Validation Loss: 2.3842973709106445
New Checkpoint Saved into PATH
Epoch 1

Epoch 241: Training Loss: 2.172278881072998 Validation Loss: 2.2978878021240234
New Checkpoint Saved into PATH
Epoch 242: Training Loss: 2.1690284411112466 Validation Loss: 2.2968502044677734
New Checkpoint Saved into PATH
Epoch 243: Training Loss: 2.169499635696411 Validation Loss: 2.295142650604248
New Checkpoint Saved into PATH
Epoch 244: Training Loss: 2.1662601629892984 Validation Loss: 2.2932040691375732
New Checkpoint Saved into PATH
Epoch 245: Training Loss: 2.166156609853109 Validation Loss: 2.2922956943511963
New Checkpoint Saved into PATH
Epoch 246: Training Loss: 2.1642128626505532 Validation Loss: 2.2912120819091797
New Checkpoint Saved into PATH
Epoch 247: Training Loss: 2.165032148361206 Validation Loss: 2.289604663848877
New Checkpoint Saved into PATH
Epoch 248: Training Loss: 2.160977602005005 Validation Loss: 2.2880454063415527
New Checkpoint Saved into PATH
Epoch 249: Training Loss: 2.159095525741577 Validation Loss: 2.2865662574768066
New Checkpoint Saved into PATH


Epoch 317: Training Loss: 2.056222915649414 Validation Loss: 2.193605422973633
New Checkpoint Saved into PATH
Epoch 318: Training Loss: 2.0524422327677407 Validation Loss: 2.1921191215515137
New Checkpoint Saved into PATH
Epoch 319: Training Loss: 2.0520478089650473 Validation Loss: 2.1902358531951904
New Checkpoint Saved into PATH
Epoch 320: Training Loss: 2.0529001553853354 Validation Loss: 2.1886959075927734
New Checkpoint Saved into PATH
Epoch 321: Training Loss: 2.0472158590952554 Validation Loss: 2.187225103378296
New Checkpoint Saved into PATH
Epoch 322: Training Loss: 2.0462734699249268 Validation Loss: 2.186049699783325
New Checkpoint Saved into PATH
Epoch 323: Training Loss: 2.0502548217773438 Validation Loss: 2.1841020584106445
New Checkpoint Saved into PATH
Epoch 324: Training Loss: 2.043562571207682 Validation Loss: 2.183279514312744
New Checkpoint Saved into PATH
Epoch 325: Training Loss: 2.0418452819188437 Validation Loss: 2.1819796562194824
New Checkpoint Saved into PAT

...

Epoch 10216: Training Loss: 0.12531563142935434 Validation Loss: 0.6649578809738159
Epoch 10217: Training Loss: 0.12525903433561325 Validation Loss: 0.6644349098205566
Epoch 10218: Training Loss: 0.12450310836235683 Validation Loss: 0.6639835238456726
Epoch 10219: Training Loss: 0.12598531693220139 Validation Loss: 0.66325843334198
Epoch 10220: Training Loss: 0.1267265429099401 Validation Loss: 0.663970947265625
Epoch 10221: Training Loss: 0.1254615286986033 Validation Loss: 0.6641506552696228
Epoch 10222: Training Loss: 0.125894695520401 Validation Loss: 0.6647405028343201
Epoch 10223: Training Loss: 0.12515352169672647 Validation Loss: 0.6645291447639465
Epoch 10224: Training Loss: 0.12467640886704127 Validation Loss: 0.6639271974563599
Epoch 10225: Training Loss: 0.12541168679793677 Validation Loss: 0.6638208031654358
Epoch 10226: Training Loss: 0.1273497467239698 Validation Loss: 0.6638380289077759
Epoch 10227: Training Loss: 0.12611614912748337 Validation Loss: 0.6639754176139832


Epoch 10316: Training Loss: 0.1254554738601049 Validation Loss: 0.6645320653915405
Epoch 10317: Training Loss: 0.1253609855969747 Validation Loss: 0.6649823188781738
Epoch 10318: Training Loss: 0.12435593704382579 Validation Loss: 0.6657247543334961
Epoch 10319: Training Loss: 0.125013567507267 Validation Loss: 0.6654784083366394
Epoch 10320: Training Loss: 0.12548009554545084 Validation Loss: 0.6650979518890381
Epoch 10321: Training Loss: 0.1244933952887853 Validation Loss: 0.6651486754417419
Epoch 10322: Training Loss: 0.126462089518706 Validation Loss: 0.664749801158905
Epoch 10323: Training Loss: 0.12496677041053772 Validation Loss: 0.6643458008766174
Epoch 10324: Training Loss: 0.12480278064807256 Validation Loss: 0.6655406951904297
Epoch 10325: Training Loss: 0.12513047208388647 Validation Loss: 0.6651840806007385
Epoch 10326: Training Loss: 0.12471096962690353 Validation Loss: 0.6648961305618286
Epoch 10327: Training Loss: 0.1247188001871109 Validation Loss: 0.6658185720443726
E

Epoch 10413: Training Loss: 0.1243479922413826 Validation Loss: 0.6660821437835693
Epoch 10414: Training Loss: 0.12384936213493347 Validation Loss: 0.6657072901725769
Epoch 10415: Training Loss: 0.12376357614994049 Validation Loss: 0.6652864813804626
Epoch 10416: Training Loss: 0.12400803218285243 Validation Loss: 0.665496826171875
Epoch 10417: Training Loss: 0.12400153776009877 Validation Loss: 0.6668401956558228
Epoch 10418: Training Loss: 0.12373749166727066 Validation Loss: 0.6667518019676208
Epoch 10419: Training Loss: 0.12389041980107625 Validation Loss: 0.6663679480552673
Epoch 10420: Training Loss: 0.1238735889395078 Validation Loss: 0.666014552116394
Epoch 10421: Training Loss: 0.12348858267068863 Validation Loss: 0.6660052537918091
Epoch 10422: Training Loss: 0.12388052046298981 Validation Loss: 0.6656464338302612
Epoch 10423: Training Loss: 0.12392223874727885 Validation Loss: 0.6658735275268555
Epoch 10424: Training Loss: 0.12376721451679866 Validation Loss: 0.6655402183532

Epoch 10512: Training Loss: 0.12362835307916005 Validation Loss: 0.6675150990486145
Epoch 10513: Training Loss: 0.12247620522975922 Validation Loss: 0.6678509712219238
Epoch 10514: Training Loss: 0.12326765805482864 Validation Loss: 0.6671252250671387
Epoch 10515: Training Loss: 0.12298338860273361 Validation Loss: 0.6667547225952148
Epoch 10516: Training Loss: 0.12261381993691127 Validation Loss: 0.6663471460342407
Epoch 10517: Training Loss: 0.1237544318040212 Validation Loss: 0.6665103435516357
Epoch 10518: Training Loss: 0.12217792123556137 Validation Loss: 0.6662358045578003
Epoch 10519: Training Loss: 0.1230096494158109 Validation Loss: 0.6664670705795288
Epoch 10520: Training Loss: 0.12300013999144237 Validation Loss: 0.6675799489021301
Epoch 10521: Training Loss: 0.12413676331440608 Validation Loss: 0.6686140298843384
Epoch 10522: Training Loss: 0.12372744580109914 Validation Loss: 0.6677743196487427
Epoch 10523: Training Loss: 0.12291467934846878 Validation Loss: 0.66682618856

KeyboardInterrupt: 

In [13]:
PATH = 'Data\\model_checkpoint.pt'
device = torch.device('cpu')
net = Net1()
net.load_state_dict(torch.load(PATH, map_location = device))

In [14]:
error = nn.CrossEntropyLoss()

In [15]:
with torch.no_grad():
    net.eval()
    inputs = torch.FloatTensor(Xtes)
    labels = torch.tensor(ytes, dtype=torch.long)
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net.forward(inputs)
    loss = error(outputs, labels) 
    print(loss.item())

0.6936106085777283


In [16]:
(0.6933146119117737, 2, 0.4)

(0.6933146119117737, 2, 0.4)

In [17]:
param_dict = {}
for v,p in enumerate(net.parameters()):
    param_dict[v] = p.data.numpy()
    
W0 = param_dict[0].T
b0 = param_dict[1]
W1 = param_dict[2].T
b1 = param_dict[3]


h0 = np.matmul(Xtes, W0) + b0
h1 = np.tanh(h0)
h2 = np.matmul(h1, W1) + b1
h3 = np.exp(h2)
o = h3/np.sum(h3,axis=1).reshape(-1,1)

In [18]:
o.shape

(228, 13)

In [19]:
np.save('Data\\param_dict.npy', param_dict)

In [20]:
def y_hat(prob, topics):    
    return topics[np.argmax(prob)]
    

In [21]:
pred_labels = []
topics = ['Jubilacion Patronal', 'Consultoria', 'Renuncia/Despido/Desahucio', 'IESS', 
                 'Greeting', 'Contacto', 'No Topic', 'Queja', 'Otros servicios', 'Charlas/Capacitaciones', 
                      'Hi Five', 'trabaje con nosotros', 'Facturacion/Retencion/Cobros']
for c in o:
    pred_labels.append(y_hat(c, topics))

In [22]:
preguntas = data.Pregunta[btes]
labels = data.Tema[btes]
ambig = data.ambiguedad[btes]

In [23]:
Xtes = np.column_stack((preguntas, labels, pred_labels, ambig, Xtes, o))
#Xtes = np.column_stack((preguntas, labels, pred_labels, Xtes, o))

In [24]:
data2 = pd.DataFrame(Xtes)


In [25]:
vocab = np.load('Data\\vocab.npy', allow_pickle=True)
colnames = np.concatenate((['pregunta', 'label', 'pred_label', 'ambig'], vocab, ['n_token', 'perc_greet', 'capacitacion', 'polarity'], ['Jubilacion Patronal', 'Consultoria', \
                                                                                                     'Renuncia/Despido/Desahucio', 'IESS', 'Greeting', \
                                                                                                         'Contacto', 'No Topic', 'Queja', 'Otros servicios', \
                                                                                                 'Charlas/Capacitaciones', 'Hi Five', 'trabaje con nosotros', 'Facturacion/Retencion/Cobros']))
data2.columns = colnames


In [26]:
data2.head(3)


,pregunta,label,pred_label,ambig,buen,dia,mi,nombr,llam,empres,...,IESS,Greeting,Contacto,No Topic,Queja,Otros servicios,Charlas/Capacitaciones,Hi Five,trabaje con nosotros,Facturacion/Retencion/Cobros
0,la empresa requiere el estudio individual de j...,Jubilacion Patronal,Jubilacion Patronal,NaN,0,0,0,0,0,1,...,0.00249933,1.79508e-05,0.000140022,0.00124754,0.00232034,0.000411475,0.00011264,7.56738e-05,1.47318e-06,8.8441e-05
1,Es lo mismo el desahucio que el despido intemp...,Renuncia/Despido/Desahucio,Renuncia/Despido/Desahucio,NaN,0,0,0,0,0,0,...,0.00015618,0.000142686,3.15172e-05,0.0001123,0.000394293,0.00151611,5.696e-05,0.00012798,0.000821461,1.37677e-05
2,"Gracias por esta tan valiosa aportacion, me gu...",Consultoria,Consultoria,NaN,0,0,0,0,0,1,...,0.00239389,3.29595e-05,0.000135439,0.00852437,0.0705995,0.014282,0.0117883,0.000277097,0.0202038,0.00268414


In [27]:
data2.to_csv('Data\\Xtes.csv', sep=',')